In [18]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score, mean_squared_error

In [19]:
import os
# Office
#os.chdir('C:\\Users\\Administrator\\Desktop\\Data\\Loan Prediction')
#data = pd.read_csv('LoanTrain.csv')

# Personal
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\Loan Prediction')
data = pd.read_csv('Loan Prediction Train.csv')

In [20]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [21]:
#### Handling the Missing Values

data.loc[data['Gender'].isnull(),'Gender']='Trasgender'
data.loc[data['Dependents'].isnull(),'Dependents']='0'
data.loc[data['Married'].isnull(),'Married']='No'
data.loc[data['Self_Employed'].isnull(),'Self_Employed']='No'
data=data.drop(index=data.loc[data['LoanAmount'].isnull(),:].index)
data.loc[data['Loan_Amount_Term'].isnull(),'Loan_Amount_Term']=0
data.loc[data['Credit_History'].isnull(),'Credit_History']=0

In [22]:
data['Credit_History']=data['Credit_History'].astype(str)

In [23]:
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
encoding_minmax=defaultdict(MinMaxScaler)
features_num=['ApplicantIncome']
for feat in features_num:
    encoded_df=pd.DataFrame()
    #d=pd.DataFrame(encoding_minmax(feat).fit_transform(new_df[feat]))
    d=pd.DataFrame(encoding_minmax[feat].fit_transform(data[[feat]]))
    test_column=d.columns.values
    list_column=[feat+'_'+'minmax' for j in test_column]
    d.columns=list_column
    encoded_df=pd.concat([encoded_df,d],axis=1).reset_index(drop=True)
    data.drop(feat,axis=1,inplace=True)
    data=data.reset_index(drop=True)
    data=pd.concat([data,encoded_df],axis=1)
import pickle
with open('C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Deployment\\Deploy_Flask\\Deploy_Flask\\LoanPrediction\\minmax_pickle.pkl','wb') as f:
    pickle.dump(encoding_minmax,f)

In [24]:
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
encoding_standardscaler=defaultdict(StandardScaler)
features_num=['LoanAmount']
for feat in features_num:
    encoded_df=pd.DataFrame()
    #d=pd.DataFrame(encoding_minmax(feat).fit_transform(new_df[feat]))
    d=pd.DataFrame(encoding_standardscaler[feat].fit_transform(data[[feat]]))
    test_column=d.columns.values
    list_column=[feat+'_'+'minmax' for j in test_column]
    d.columns=list_column
    encoded_df=pd.concat([encoded_df,d],axis=1).reset_index(drop=True)
    data.drop(feat,axis=1,inplace=True)
    data=data.reset_index(drop=True)
    data=pd.concat([data,encoded_df],axis=1)
with open('C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Deployment\\Deploy_Flask\\Deploy_Flask\\LoanPrediction\\standardscaler_pickle.pkl','wb') as f:
    pickle.dump(encoding_standardscaler,f)

In [25]:
data.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'CoapplicantIncome', 'Loan_Amount_Term',
       'Credit_History', 'Property_Area', 'Loan_Status',
       'ApplicantIncome_minmax', 'LoanAmount_minmax'],
      dtype='object')

In [26]:
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
encoding_binarizer=defaultdict(LabelBinarizer)
col_nominal=['Married','Gender']
for feat in col_nominal:
    encoded_df=pd.DataFrame()
    d=pd.DataFrame(encoding_binarizer[feat].fit_transform(data[feat]))
    test_column=d.columns.values
    list_columns=[feat+'_'+str(j) for j in test_column]
    #print(list_columns)
    d.columns=list_columns
    #print(d.head())
    encoded_df=pd.concat([encoded_df,d],axis=1).reset_index(drop=True)
    data.drop(feat,axis=1,inplace=True)
    data=data.reset_index(drop=True)
    data=pd.concat([data,encoded_df],axis=1)
with open('C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Deployment\\Deploy_Flask\\Deploy_Flask\\LoanPrediction\\binarizer_pickle.pkl','wb') as f:
    pickle.dump(encoding_binarizer,f)

In [27]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
encoding_labelencoder=defaultdict(LabelEncoder)
col_ordinal=['Education','Property_Area','Loan_Status','Dependents','Credit_History']
for feat in col_ordinal:
    encoded_df=pd.DataFrame()
    d=pd.DataFrame(encoding_labelencoder[feat].fit_transform(data[feat]))
    test_column=d.columns.values
    list_columns=[feat+'_'+str(j) for j in test_column]
    d.columns=list_columns
    encoded_df=pd.concat([encoded_df,d],axis=1).reset_index(drop=True)
    data.drop(feat,axis=1,inplace=True)
    data=data.reset_index(drop=True)
    data=pd.concat([data,encoded_df],axis=1)
with open('C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Deployment\\Deploy_Flask\\Deploy_Flask\\LoanPrediction\\labelencoder_pickle.pkl','wb') as f:
    pickle.dump(encoding_labelencoder,f)

In [28]:
data=data.drop(['Loan_ID','Self_Employed'],axis=1)

In [29]:
data.head()

,CoapplicantIncome,Loan_Amount_Term,ApplicantIncome_minmax,LoanAmount_minmax,Married_0,Gender_0,Gender_1,Gender_2,Education_0,Property_Area_0,Loan_Status_0,Dependents_0,Credit_History_0
0,1508.0,360.0,0.054830,-0.215309,1,0,1,0,0,0,0,1,1
1,0.0,360.0,0.035250,-0.940328,1,0,1,0,0,2,1,0,1
2,2358.0,360.0,0.030093,-0.308860,1,0,1,0,1,2,1,0,1
3,0.0,360.0,0.072356,-0.063289,0,0,1,0,0,2,1,0,1
4,4196.0,360.0,0.065145,1.410137,1,0,1,0,0,2,1,2,1


In [30]:
data.dtypes

CoapplicantIncome         float64
Loan_Amount_Term          float64
ApplicantIncome_minmax    float64
LoanAmount_minmax         float64
Married_0                   int32
Gender_0                    int32
Gender_1                    int32
Gender_2                    int32
Education_0                 int32
Property_Area_0             int32
Loan_Status_0               int32
Dependents_0                int32
Credit_History_0            int32
dtype: object

#### Train Test Split

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data.drop('Loan_Status_0',axis=1),data['Loan_Status_0'],test_size=0.30,random_state=42)
X_train.shape, X_test.shape

((414, 12), (178, 12))

In [32]:
data.head()

,CoapplicantIncome,Loan_Amount_Term,ApplicantIncome_minmax,LoanAmount_minmax,Married_0,Gender_0,Gender_1,Gender_2,Education_0,Property_Area_0,Loan_Status_0,Dependents_0,Credit_History_0
0,1508.0,360.0,0.054830,-0.215309,1,0,1,0,0,0,0,1,1
1,0.0,360.0,0.035250,-0.940328,1,0,1,0,0,2,1,0,1
2,2358.0,360.0,0.030093,-0.308860,1,0,1,0,1,2,1,0,1
3,0.0,360.0,0.072356,-0.063289,0,0,1,0,0,2,1,0,1
4,4196.0,360.0,0.065145,1.410137,1,0,1,0,0,2,1,2,1


In [37]:
import pickle
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)

#with open('C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Deployment\\Deploy_Flask\\Deploy_Flask\\LoanPrediction\\LogisticRegression_MODEL.pkl','wb') as f:
pickle_out = open("C:\\Users\\prudi\\Documents\\GitHub\\Code\\2019\\Deployment\\Deploy_Flask\\Deploy_Flask\\LoanPrediction\\LogisticRegression_MODEL.pkl","wb")
pickle.dump(lr, pickle_out)
pickle_out.close()

